In [79]:
import os, re
import pandas as pd
import numpy as np
from konlpy.tag import Okt

In [4]:
PATH='/root/workspace/data/'
event=pd.read_csv(PATH+'reviewevent.csv', encoding='utf-8')

In [6]:
event

,shop,notice,isservice,reviewevent
0,JH양동시장통닭-중랑점,1984년 처음 양동시장에서 장사를 시작하고나서\n2017년 아들내외와 같이 이곳 ...,False,NaN
1,본초불닭발-면목점,👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1 드...,True,\n👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1...
2,호치킨-전농점,★ 신메뉴 / 빠다갈릭치킨 / 출시 기념 ‘ 빠삭꾸이포 ’ 증정 이벤트 (•ө•) ...,True,\n★ 신메뉴 / 빠다갈릭치킨 / 출시 기념 ‘ 빠삭꾸이포 ’ 증정 이벤트 (•ө•...
3,석도명품닭갈비-본점,안녕하십니까 석도대표입니다\n저희매장은 닭으로 관련된 메뉴들로 구성되어 있는데요\n...,True,\n■■■■리뷰이벤트■■■■\n이벤트 참여시 우동사리or치즈사리 서비스드립니다\n요...
4,청년육갑-고대점,56.7도의 마법 (수비드 삼겹살)\n\n배달 최초!!! 수비드(저온조리) 삼겹살을...,True,\n오픈 이벤트를 진행합니다!\n1. 3만원 이상 구매시 막국수 서비스!!\n4. ...
...,...,...,...,...
927,까로치아-용두점,"☜(ﾟヮﾟ☜)공지 사항 확인해주세요(☞ﾟヮﾟ)☞\n\n길이 빙판길라서 당분간 길음,...",False,NaN
928,새서울청과,*상단에 있는 찜을 눌러주시고 바로결제 하신 분들에 한하여 서비스를 제공합니다.\n...,True,\n*상단에 있는 찜을 눌러주시고 바로결제 하신 분들에 한하여 서비스를 제공합니다....
929,CAFE153,‼️요기요 익스프레스는 라이더 배정 후 저희가 주문 확인 가능합니다. 라이더가 배정...,False,NaN
930,커피빈-장안점,안녕하세요.\n커피빈을 찾아주시는 고객님을 위해 '사진 리뷰 이벤트'를 진행합니다....,True,\n커피빈을 찾아주시는 고객님을 위해 '사진 리뷰 이벤트'를 진행합니다.\n이벤트에...


In [67]:
event['photo']=event.reviewevent.map(lambda x: extract_photo(x))

In [68]:
pd.set_option('display.max_colwidth', None)
event[event.photo.notnull()]

,shop,notice,isservice,reviewevent,photo
1,본초불닭발-면목점,👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1 드립니다\n\n1. 콜라500ml\n2. 사이다500ml\n3. 콘버터\n\n💕리뷰이벤트 참여 방법💕\n1⃣ 먼저 우리가게를 찜 ❤해주세요\n2⃣ 요청사항에 [리뷰참여 택1메뉴선택]\n3⃣ 맛있게 먹고 사진과 리뷰,True,\n👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1 드립니다\n💕리뷰이벤트 참여 방법💕\n1⃣ 먼저 우리가게를 찜 ❤해주세요\n2⃣ 요청사항에 [리뷰참여 택1메뉴선택]\n3⃣ 맛있게 먹고 사진과 리뷰,3⃣ 맛있게 먹고 사진과 리뷰
2,호치킨-전농점,"★ 신메뉴 / 빠다갈릭치킨 / 출시 기념 ‘ 빠삭꾸이포 ’ 증정 이벤트 (•ө•) ♡★ \n\n고소한 버터크림과 달다란 마늘의 부드러운 풍미, 마늘 바게트와 함께 즐기는 ★ 빠다갈릭치킨 ★ 출시~!! (짝짝짝)\n\n▶ 신메뉴 ＇빠다갈릭치킨‘ 주문하면, ‘ 빠삭꾸이포＇ 증정! (단, 내맘대로두마리 메뉴 제외)\n\n★ 서비스는 사랑입니다♡ ◟( ˘ ³˘)◞ ★\n- 치킨 단품 메뉴: 코카콜라 245ml 서비스\n- 치킨 SET 메뉴: 코카콜라 500ml 서비스\n\n★ 치즈볼 증정 리뷰이벤트 진행중!\n▶ 리뷰 이벤트 참여방법 ◀\n1. 단골가게 찜! ❤\n2. 바로결제 (만나서결제 고객님제외)\n3. 요청사항란에 ex)""사진 리뷰 약속” 남기면 끝!\n\n▶ 치킨과 치즈볼 맛있게 드셨다면 ★별★다★섯★개★와 함께 기분 좋은 리뷰! 꼭! 부탁드려요~ ❤\n‘바로결제’만 가능한 점 양해 부탁드립니다!\n\n★ 내 삶의 작은 변화, 1회용품 줄이기 ★ 나무젓가락은 기본제공이 되지 않습니다. 필요시 요청란에 남겨주세요~",True,"\n★ 신메뉴 / 빠다갈릭치킨 / 출시 기념 ‘ 빠삭꾸이포 ’ 증정 이벤트 (•ө•) ♡★ \n▶ 신메뉴 ＇빠다갈릭치킨‘ 주문하면, ‘ 빠삭꾸이포＇ 증정! (단, 내맘대로두마리 메뉴 제외)\n★ 서비스는 사랑입니다♡ ◟( ˘ ³˘)◞ ★\n- 치킨 단품 메뉴: 코카콜라 245ml 서비스\n- 치킨 SET 메뉴: 코카콜라 500ml 서비스\n★ 치즈볼 증정 리뷰이벤트 진행중!\n▶ 리뷰 이벤트 참여방법 ◀\n1. 단골가게 찜! ❤\n2. 바로결제 (만나서결제 고객님제외)\n3. 요청사항란에 ex)""사진 리뷰 약속” 남기면 끝!\n▶ 치킨과 치즈볼 맛있게 드셨다면 ★별★다★섯★개★와 함께 기분 좋은 리뷰! 꼭! 부탁드려요~ ❤\n‘바로결제’만 가능한 점 양해 부탁드립니다!\n★ 내 삶의 작은 변화, 1회용품 줄이기 ★ 나무젓가락은 기본제공이 되지 않습니다. 필요시 요청란에 남겨주세요~","3. 요청사항란에 ex)""사진 리뷰 약속” 남기면 끝!"
9,치킨먹9회두먹9,🎁치킨먹9회도먹9 리뷰이벤트🎁\n\n요청사항에 리뷰이벤트참여와 서비스상품을 선택하여 적어주세요!\n\n닭강정or닭똥집 중 택1\n\n❤맛있게 드신 후 찜등록과 별5개 포토리뷰 부탁드립니다❤,True,\n🎁치킨먹9회도먹9 리뷰이벤트🎁\n요청사항에 리뷰이벤트참여와 서비스상품을 선택하여 적어주세요!\n❤맛있게 드신 후 찜등록과 별5개 포토리뷰 부탁드립니다❤,❤맛있게 드신 후 찜등록과 별5개 포토리뷰 부탁드립니다❤
14,BHC-장안점,"리뷰이벤트 진행중이니 확인하시고 꼭 참여하세요\n서비스와 함께 더욱더 맛있게 드세요❤\n\nBHC 우수가맹점의 자부심으로 고객님께\n행복한 치킨과 혜택을 드리겠습니다!\n\n♥♥♥맛있는 치킨먹고 서비스받기♥♥♥\n♥1. 상단에 '찜' 하트 누르고 단골매장 등록~♥\n♥2. 주문 후 요청사항에 '리뷰이벤트ㅇㅇ주세요' 등록~♥\n♥3. 치킨 받고 사진, 별점, 예쁜리뷰 작성~♥\n♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥\n\n무더운 여름 시원 시원하게 리뷰 품목을 늘려봤습니다!!\n아래 품목중 1택 입니다!!\n\n★ 코 카 콜 라 1.25L SIZE UP !\n★ 뿌 링 치 즈 스 틱 3개 증정 !\n★ 빨간소떡 or 뿌링소떡 증정 !\n★ 뿌링 소스 or 커리퀸 딥소스 !\n★ 한입 쏙쏙 미니 콜팝치킨 증정 !\n\n1. 상단에 ❤ 찜버튼 눌러주세요!\n2. 주문후 요청사항에\n(리뷰이벤트 ㅇㅇ주세요~) 적어주세요!\n3. 맛있게 드신후 꼭! 예쁜사진과 함께\n별점 5점 소중한 리뷰 부탁 드려요!\n\n※ 치즈스틱, 콜팝은 치킨과 함께 토핑으로 보내 드립니다~\n\n♥BHC장안점 쿠폰 이벤트♥\n\n→ 쿠폰 3매 : 감자튀김 or 치즈스틱 4개 !!!\n\n→ 쿠폰 6매 : 달콤바삭치즈볼 or 생맥주 1000cc !!!\n\n→ 쿠폰 13매 : 후라이드 한마리 !!!\n\n※명함쿠폰 사용시 매장으로 말씀 해주시면 서비스와 함께 친절히 설명 드리겠습니다~♥\n\n※ '스윗텐더' 주문시 소스는 '뿌링뿌링소스' 로 대체해서 보내 드립니다~^^❤\n\n♥100% 국내산 닭고기, 매일 정제한 깨끗한 기름,\n당일 손질한 신선한 재료만 사용합니다!\n한정된 물량 조진시 조기종료 될수있습니다!\n\n♥선 조리 없이 주문 즉시 조리하여 시간이\n다소 소요됩니다. 또한 주문 즉시 조리하기\n때문에 주문 접수시 취소가 불가합니다!\n\n♥배달지역♥\n장안동, 답십리동, 전농동, 용답동, 휘경동, 중곡동,\n면목동, 이문동, 청량리동, 회기동\n\n♥항상 최고의 서비스를 드리기 위해 노력하는\nBHC 장안점이 되겠습니다! 감사합니다~^^♥\n\n♥조금만 기다려 주세요! '행복배달꾼'이 맛있는\n치킨을 안전하게 보내 드리겠습니다~^^♥\n\n※ 본 업소는 코로나19 확산 초기부터 현재까지 매일 방역과 위생에 힘쓰고 있습니다.",True,"\n리뷰이벤트 진행중이니 확인하시고 꼭 참여하세요\n서비스와 함께 더욱더 맛있게 드세요❤\n♥♥♥맛있는 치킨먹고 서비스받기♥♥♥\n♥1. 상단에 '찜' 하트 누르고 단골매장 등록~♥\n♥2. 주문 후 요청사항에 '리뷰이벤트ㅇㅇ주세요' 등록~♥\n♥3. 치킨 받고 사진, 별점, 예쁜리뷰 작성~♥\n무더운 여름 시원 시원하게 리뷰 품목을 늘려봤습니다!!\n★ 뿌 링 치 즈 스 틱 3개 증정 !\n★ 빨간소떡 or 뿌링소떡 증정 !\n★ 한입 쏙쏙 미니 콜팝치킨 증정 !\n1. 상단에 ❤ 찜버튼 눌러주세요!\n2. 주문후 요청사항에\n(리뷰이벤트 ㅇㅇ주세요~) 적어주세요!\n3. 맛있게 드신후 꼭! 예쁜사진과 함께\n별점 5점 소중한 리뷰 부탁 드려요!\n♥BHC장안점 쿠폰 이벤트♥\n※명함쿠폰 사용시 매장으로 말씀 해주시면 서비스와 함께 친절히 설명 드리겠습니다~♥\n♥항상 최고의 서비스를 드리기 위해 노력하는","♥3. 치킨 받고 사진, 별점, 예쁜리뷰 작성~♥\n3. 맛있게 드신후 꼭! 예쁜사진과 함께"
19,멕시칸치킨-답십리장안점,찜버튼#리뷰$사진이벤트#\n올한해 많은 성원과 관심에 보답해드리고자 이벤트 메뉴를 새롭게 추가하였습니다.\n1. 미니핫도그\n2. 새우튀김\n3. 치즈스틱\n4. 감자볼\n(넷 중 1 선택)\n많은 참여와 관심 부탁드립니다.,True,\n찜버튼#리뷰$사진이벤트#\n올한해 많은 성원과 관심에 보답해드리고자 이벤트 메뉴를 새롭게 추가하였습니다.\n(넷 중 1 선택)\n많은 참여와 관심 부탁드립니다.,찜버튼#리뷰$사진이벤트#
...,...,...,...,...,...
904,굿샌드,♡♡안녕하세요♡♡\n이번에 새롭게 오픈한 굿샌드 입니다\n다양하고신선한 샐러드 샌드위치 커피 음료수 전문매장 입니다\n♡♡포토리뷰이벤트를 진행합니다♡♡\n♡♡복숭아아이스티♡♡\n♡♡해쉬브라운감자2개♡♡\n둘중하나를 선택해서 신청해주세요. 감사합니다♡♡,True,\n♡♡포토리뷰이벤트를 진행합니다♡♡\n둘중하나를 선택해서 신청해주세요. 감사합니다♡♡,♡♡포토리뷰이벤트를 진행합니다♡♡
910,봉명동내커피-경희대점,★★봉명동 내커피 서울 첫 OPEN ★★\n합리적 가격에 믿고 드실 수 있는 봉.내커피\n드디어 서울에도 상륙했습니다!\n\n❤❤오픈 기념 초대박 리뷰 이벤트❤❤\n\n-찜♥ + 별5개 + 포토리뷰 작성을 약속해 주시는 모든분들께\n모든 음료를 큰 사이즈로 보내드립니다😆\n-요청사항에 꼭! **닉네임**과 함께 리뷰 참여한다고\n기재 부탁드

In [66]:
def extract_photo(sent):
    if sent != sent :
        return None
    extract=None
    sent_split=sent.split('\n')
    find=['사진','포토']
    for sent in sent_split:
        for word in find:
            if re.search(word,sent):
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract
            

In [52]:
sent=event.reviewevent[1]

In [53]:
sent

'\n👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1 드립니다\n💕리뷰이벤트 참여 방법💕\n1⃣ 먼저 우리가게를 찜 ❤해주세요\n2⃣ 요청사항에 [리뷰참여 택1메뉴선택]\n3⃣ 맛있게 먹고 사진과 리뷰'

In [81]:
okt=Okt()
okt.morphs(sent)

['\n',
 '👉',
 'OPEN',
 '기념',
 '리뷰',
 '이벤트',
 '👈',
 '\n',
 '소중한',
 '리뷰',
 '를',
 '남겨주시는',
 '고객',
 '님',
 '께',
 '서비스',
 '메뉴',
 '택',
 '1',
 '드립니다',
 '\n',
 '💕',
 '리뷰',
 '이벤트',
 '참여',
 '방법',
 '💕',
 '\n',
 '1',
 '⃣',
 '먼저',
 '우리',
 '가게',
 '를',
 '찜',
 '❤',
 '해주세요',
 '\n',
 '2',
 '⃣',
 '요청',
 '사항',
 '에',
 '[',
 '리뷰',
 '참여',
 '택',
 '1',
 '메뉴',
 '선택',
 ']',
 '\n',
 '3',
 '⃣',
 '맛있게',
 '먹고',
 '사진',
 '과',
 '리뷰']

In [54]:
extract_photo(sent)

'3⃣ 맛있게 먹고 사진과 리뷰'

In [72]:
pd.set_option('display.max_rows', None)
event.photo[event.photo.notnull()]

1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    3⃣ 맛있게 먹고 사진과 리뷰
2                                                                                                                                                                                                                                                                                                                                                 

In [76]:
def extract_star(sent):
    if sent != sent :
        return None
    extract=None
    sent_split=sent.split('\n')
    find=['별']
    for sent in sent_split:
        for word in find:
            if re.search(word,sent):
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [77]:
event['star']=event.reviewevent.map(lambda x: extract_star(x))

In [78]:
event['star']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     None
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [82]:
def extract_star2(sent):
    if sent != sent :
        return None
    
    extract=None
    sent_split=sent.split('\n')
    find=['별']
    for sent in sent_split:
        token=okt.morphs(sent)
        for word in find:
            if word in token:
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [83]:
event['star']=event.reviewevent.map(lambda x: extract_star2(x))

In [85]:
len(event.star[event.star.notnull()])

159

In [86]:
def extract_nickname(sent):
    if sent != sent :
        return None
    extract=None
    sent_split=sent.split('\n')
    find=['닉네임','아이디']
    for sent in sent_split:
        for word in find:
            if re.search(word,sent):
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [88]:
event['nickname']=event.reviewevent.map(lambda x: extract_nickname(x))

In [90]:
len(event.nickname[event.nickname.notnull()])

147

In [230]:
def extract_jimm(sent):
    if sent != sent :
        return None
    
    extract=None
    sent_split=sent.split('\n')
    find=['찜']
    findx=['계란찜','갈비찜','아구찜','해물찜','찜닭','찜탕','두찜','행찜','뼈다귀찜']
    for sent in sent_split:
        if re.search(find[0],sent):
            cnt=0
            for word in findx:
                if re.search(word,sent):
                    break
            else:
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
    return extract

In [231]:
event['jimm']=event.reviewevent.map(lambda x: extract_jimm(x))

In [232]:
len(event.jimm[event['jimm'].notnull()])

222

In [250]:
event.head()

,shop,notice,isservice,reviewevent,photo,star,nickname,jimm,photo_t,star_t,nickname_t,jimm_t
0,JH양동시장통닭-중랑점,"1984년 처음 양동시장에서 장사를 시작하고나서\n2017년 아들내외와 같이 이곳 중화동에 올라왔읍니다.\n제가 여기 서울에 올라오니 이쪽 음식은 짜고 자극적인 것들이 많읍니다.\n저희 음식은 자극적이지 않고 기본에 충실한 음식들입니다.\n그래서 아직도 많은 손님들이 찾아주시지 않나 싶읍니다.\n닭은 제 아들이 매일 아침에 트럭을 몰고 양계장에서 직접 사오고 있으며,\n상태가 안좋은 닭은 제가 매일 확인해서 폐기처분하고 있읍니다.\n모든 소스는 제가 직접 만들고 있읍니다. 다른곳에서 찾기 힘든 특별한 소스이니 먹어보시면 좋을겁니다.\n지금도 그때의 단골손님들이 기억하시고 이따금 저를 찾아오시곤 합니다. 자제분들의 자제분들도 저희 음식을 먹고 있읍니다.\n최근 손녀가 태어났읍니다. 제 손녀한테도 안심하고 먹일수 있는 음식을 만들겠읍니다.\n늘 소통하겠읍니다.\n감사합니다.",False,NaN,None,None,None,None,False,False,False,False
1,본초불닭발-면목점,👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1 드립니다\n\n1. 콜라500ml\n2. 사이다500ml\n3. 콘버터\n\n💕리뷰이벤트 참여 방법💕\n1⃣ 먼저 우리가게를 찜 ❤해주세요\n2⃣ 요청사항에 [리뷰참여 택1메뉴선택]\n3⃣ 맛있게 먹고 사진과 리뷰,True,\n👉OPEN기념 리뷰 이벤트👈\n소중한리뷰를 남겨주시는 고객님께 서비스 메뉴 택1 드립니다\n💕리뷰이벤트 참여 방법💕\n1⃣ 먼저 우리가게를 찜 ❤해주세요\n2⃣ 요청사항에 [리뷰참여 택1메뉴선택]\n3⃣ 맛있게 먹고 사진과 리뷰,3⃣ 맛있게 먹고 사진과 리뷰,None,None,1⃣ 먼저 우리가게를 찜 ❤해주세요,True,False,False,True
2,호치킨-전농점,"★ 신메뉴 / 빠다갈릭치킨 / 출시 기념 ‘ 빠삭꾸이포 ’ 증정 이벤트 (•ө•) ♡★ \n\n고소한 버터크림과 달다란 마늘의 부드러운 풍미, 마늘 바게트와 함께 즐기는 ★ 빠다갈릭치킨 ★ 출시~!! (짝짝짝)\n\n▶ 신메뉴 ＇빠다갈릭치킨‘ 주문하면, ‘ 빠삭꾸이포＇ 증정! (단, 내맘대로두마리 메뉴 제외)\n\n★ 서비스는 사랑입니다♡ ◟( ˘ ³˘)◞ ★\n- 치킨 단품 메뉴: 코카콜라 245ml 서비스\n- 치킨 SET 메뉴: 코카콜라 500ml 서비스\n\n★ 치즈볼 증정 리뷰이벤트 진행중!\n▶ 리뷰 이벤트 참여방법 ◀\n1. 단골가게 찜! ❤\n2. 바로결제 (만나서결제 고객님제외)\n3. 요청사항란에 ex)""사진 리뷰 약속” 남기면 끝!\n\n▶ 치킨과 치즈볼 맛있게 드셨다면 ★별★다★섯★개★와 함께 기분 좋은 리뷰! 꼭! 부탁드려요~ ❤\n‘바로결제’만 가능한 점 양해 부탁드립니다!\n\n★ 내 삶의 작은 변화, 1회용품 줄이기 ★ 나무젓가락은 기본제공이 되지 않습니다. 필요시 요청란에 남겨주세요~",True,"\n★ 신메뉴 / 빠다갈릭치킨 / 출시 기념 ‘ 빠삭꾸이포 ’ 증정 이벤트 (•ө•) ♡★ \n▶ 신메뉴 ＇빠다갈릭치킨‘ 주문하면, ‘ 빠삭꾸이포＇ 증정! (단, 내맘대로두마리 메뉴 제외)\n★ 서비스는 사랑입니다♡ ◟( ˘ ³˘)◞ ★\n- 치킨 단품 메뉴: 코카콜라 245ml 서비스\n- 치킨 SET 메뉴: 코카콜라 500ml 서비스\n★ 치즈볼 증정 리뷰이벤트 진행중!\n▶ 리뷰 이벤트 참여방법 ◀\n1. 단골가게 찜! ❤\n2. 바로결제 (만나서결제 고객님제외)\n3. 요청사항란에 ex)""사진 리뷰 약속” 남기면 끝!\n▶ 치킨과 치즈볼 맛있게 드셨다면 ★별★다★섯★개★와 함께 기분 좋은 리뷰! 꼭! 부탁드려요~ ❤\n‘바로결제’만 가능한 점 양해 부탁드립니다!\n★ 내 삶의 작은 변화, 1회용품 줄이기 ★ 나무젓가락은 기본제공이 되지 않습니다. 필요시 요청란에 남겨주세요~","3. 요청사항란에 ex)""사진 리뷰 약속” 남기면 끝!",▶ 치킨과 치즈볼 맛있게 드셨다면 ★별★다★섯★개★와 함께 기분 좋은 리뷰! 꼭! 부탁드려요~ ❤,None,1. 단골가게 찜! ❤,True,True,False,True
3,석도명품닭갈비-본점,안녕하십니까 석도대표입니다\n저희매장은 닭으로 관련된 메뉴들로 구성되어 있는데요\n이는 제가 15년째 현역 닭유통 사업을 같이하기 때문입니다 무엇보다 제가 자신있는 분야로 소비자분들께서 보다좋은재료로 가족.지인.친구들과 건강한 식사대접을 위해 노력하겠습니다\n최고보단 최선을 다하겠습니다\n\n■■■■리뷰이벤트■■■■\n이벤트 참여시 우동사리or치즈사리 서비스드립니다\n요청사항란에 리뷰이벤트 참여합니다 적어주시고\n필요하신 사리 택해서 적어주시면 됩니다\n많은 관심부탁드립니다 감사합니다,True,\n■■■■리뷰이벤트■■■■\n이벤트 참여시 우동사리or치즈사리 서비스드립니다\n요청사항란에 리뷰이벤트 참여합니다 적어주시고,None,None,None,None,False,False,False,False
4,청년육갑-고대점,56.7도의 마법 (수비드 삼겹살)\n\n배달 최초!!! 수비드(저온조리) 삼겹살을 배달해드립니다\n\n72시간 워터에이징과 48시간 드라이에이징으로 최상의 고기 품질을 만듭니다.\n5년 동안 고기집을 운영하면서 생긴 노하우로 고기를 연구했습니다.\n\n오픈 이벤트를 진행합니다!\n1. 3만원 이상 구매시 막국수 서비스!!\n4. 모든 구매 주문 고객 휴대용 소독용품 증정!!\n\n고대 공유주방에 입점하여 오프라인 매장을 동시에 운영중이며\n오픈 주방 특성상 위생에 철저히 신경쓰고 있습니다.,True,\n오픈 이벤트를 진행합니다!\n1. 3만원 이상 구매시 막국수 서비스!!\n4. 모든 구매 주문 고객 휴대용 소독용품 증정!!,None,None,None,None,False,False,False,False


In [257]:
event_t=event.iloc[:,[0,1,2,3,8,9,10,11]]

In [258]:
event_t.to_csv(PATH+'category_t.csv')

In [235]:
#event.to_csv(PATH+'category.csv')

In [239]:
def change(x):
    if x==None :
        return False
    else:
        return True
    

In [246]:
event['photo_t']=event.photo.map(lambda x: change(x))
event['star_t']=event.star.map(lambda x: change(x))


In [249]:
event['nickname_t']=event.nickname.map(lambda x: change(x))
event['jimm_t']=event.jimm.map(lambda x: change(x))

In [233]:
event.jimm[event['jimm'].notnull()]

1                                                                                                                                                                                                                                   1⃣ 먼저 우리가게를 찜 ❤해주세요
2                                                                                                                                                                                                                                          1. 단골가게 찜! ❤
9                                                                                                                                                                                                                       ❤맛있게 드신 후 찜등록과 별5개 포토리뷰 부탁드립니다❤
11                                                                                                                                                                                                                       별⭐⭐⭐⭐⭐점과 찜❤꼭~눌러 주시는거 잊지마시구요.^^
12      

In [131]:
sent=event.reviewevent[400]
sent=sent.split('\n')

In [132]:
sent

['',
 '❤️강명수 묵은지김치찜을❤️ 믿고 주문해주시는 모든 고객님께 감사드립니다.',
 '*저희 강명수 묵은지찜의 삼겹살과 앞다리살은 스페인갈라시아지방에서의 알밤을 먹고',
 '🔥사장님께서 힘든 모두를 위해 특별한 리뷰이벤트🔥',
 '즐겨찾기(❤️)를 누르시고 주문하실 음식을 고르신 후 요청사항에',
 '⭐️"닉네임과 사진 리뷰 남길께요~"와 함께',
 '받고싶은 서비스 선택!⭐️',
 '받으신 음식의 사진과 리뷰 부탁드립니다!!',
 '(주문시 마다 참여 가능합니다)',
 '소중한 리뷰 감사드리며 늘 건강하시고 행복한 하루되세요!!']

In [194]:
def extract_jimm2(sent):
    if sent != sent :
        return None
    
    extract=None
    sent_split=sent.split('\n')
    find=['찜']
    for sent in sent_split:
        if re.search(find[0],sent):
            clean_sent=re.sub('[^가-힣]',' ',sent)
            if '찜'in okt.morphs(clean_sent):
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
        
    return extract